# EfficientNet 

## EfficientNet X

In [6]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# 시드 고정
seed = 2021
deterministic = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, test_loss):
        if self.best_loss is None or test_loss < self.best_loss - self.delta:
            self.best_loss = test_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 테스트)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)

# 기존 FC Layer의 입력 차원 확인
in_features = model._fc.in_features  # EfficientNet의 FC 입력 차원

# 새로운 classifier 정의
model._fc = nn.Sequential(
    nn.Linear(in_features, 512),   # 기존 in_features에서 512로 축소
    nn.BatchNorm1d(512),           # 배치 정규화 추가
    nn.ReLU(),                     # 활성화 함수
    nn.Dropout(0.5),               # 드롭아웃 추가
    nn.Linear(512, num_classes)     # 최종 출력 (num_classes)
)

# 모델을 GPU/CPU로 이동
model.to(device)

print("Using device:", device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100
early_stopping = EarlyStopping(patience=10)

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "./best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


Loaded pretrained weights for efficientnet-b0
Using device: cuda

[Epoch 1/100]


Train Loss: 1.2062 | Train Acc: 0.6215 | Test Loss: 1.5193 | Test Acc: 0.4463
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 0.7022 | Train Acc: 0.8941 | Test Loss: 1.3109 | Test Acc: 0.6497
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.6180 | Train Acc: 0.9421 | Test Loss: 1.0197 | Test Acc: 0.7910
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.5834 | Train Acc: 0.9647 | Test Loss: 0.9354 | Test Acc: 0.8023
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.4827 | Train Acc: 0.9873 | Test Loss: 0.7478 | Test Acc: 0.8644

[Epoch 21/100]


Train Loss: 0.4949 | Train Acc: 0.9859 | Test Loss: 0.7673 | Test Acc: 0.8757

[Epoch 22/100]


Train Loss: 0.4972 | Train Acc: 0.9845 | Test Loss: 0.7493 | Test Acc: 0.8701

[Epoch 23/100]


Train Loss: 0.4963 | Train Acc: 0.9873 | Test Loss: 0.7604 | Test Acc: 0.8701

[Epoch 24/100]


Train Loss: 0.5677 | Train Acc: 0.9477 | Test Loss: 0.8541 | Test Acc: 0.8475
Early Stopping Triggered!

Best Accuracy: 0.9096


## EfficientNet O

In [7]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# ✅ 시드 고정
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ 하이퍼파라미터 설정
num_classes = 6
batch_size = 32
learning_rate = 0.001  # VGG19의 안정적인 학습을 위해 낮은 lr 사용
image_size = 224  # VGG19의 입력 크기

# ✅ 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ 데이터셋 로드
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# ✅ 데이터셋 분할 (80% 훈련, 20% 테스트)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# ✅ DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ✅ VGG19 모델 불러오기 (EfficientNet과 유사한 방식 적용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ✅ 최신 PyTorch 권장 방식으로 사전 학습된 가중치 로드
model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)  # ✅ 최신 방식 사용
# ✅ 마지막 분류기 수정 (EfficientNet처럼 사용)
model.classifier[6] = nn.Sequential(
            nn.Linear(4096, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
model = model.to(device)


print("Using device:", device)

# ✅ 손실 함수 및 최적화 함수 정의 (EfficientNet과 동일한 방식 적용)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100
early_stopping = EarlyStopping(patience=10)

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "./best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


Using device: cuda

[Epoch 1/100]


Train Loss: 1.8374 | Train Acc: 0.2825 | Test Loss: 6.0821 | Test Acc: 0.2034
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 1.7016 | Train Acc: 0.3432 | Test Loss: 1.8021 | Test Acc: 0.2655
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 1.6368 | Train Acc: 0.3545 | Test Loss: 1.4539 | Test Acc: 0.4294
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 1.6347 | Train Acc: 0.3658 | Test Loss: 1.4503 | Test Acc: 0.4237

[Epoch 5/100]


...

Train Loss: 0.7325 | Train Acc: 0.8771 | Test Loss: 1.3259 | Test Acc: 0.6328

[Epoch 40/100]


Train Loss: 0.7473 | Train Acc: 0.8686 | Test Loss: 1.5171 | Test Acc: 0.5424

[Epoch 41/100]


Train Loss: 0.8477 | Train Acc: 0.8291 | Test Loss: 1.2684 | Test Acc: 0.6610

[Epoch 42/100]


Train Loss: 0.8474 | Train Acc: 0.8220 | Test Loss: 1.3693 | Test Acc: 0.5876

[Epoch 43/100]


Train Loss: 0.6968 | Train Acc: 0.8842 | Test Loss: 1.3331 | Test Acc: 0.6328
Early Stopping Triggered!

Best Accuracy: 0.6610


In [5]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Hybrid Model (EfficientNet + VGG19)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()
        
        # ✅ VGG19 사용
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG19 출력 채널

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.vgg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Linear(efficientnet_out_features + vgg_out_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = eff_features.view(eff_features.size(0), -1)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = vgg_features.view(vgg_features.size(0), -1)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)
        return self.classifier(fused_features)

# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 1.0503 | Train Acc: 0.6158 | Val Loss: 1.4668 | Val Acc: 0.5876
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 0.3651 | Train Acc: 0.8927 | Val Loss: 0.8040 | Val Acc: 0.7797
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.1942 | Train Acc: 0.9449 | Val Loss: 0.5780 | Val Acc: 0.8136
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.1545 | Train Acc: 0.9520 | Val Loss: 0.6979 | Val Acc: 0.8305
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.1582 | Train Acc: 0.9590 | Val Loss: 0.9919 | Val Acc: 0.7062

[Epoch 18/100]


Train Loss: 0.1643 | Train Acc: 0.9590 | Val Loss: 0.8751 | Val Acc: 0.7571

[Epoch 19/100]


Train Loss: 0.1757 | Train Acc: 0.9534 | Val Loss: 2.5621 | Val Acc: 0.5198

[Epoch 20/100]


Train Loss: 0.1871 | Train Acc: 0.9463 | Val Loss: 0.8008 | Val Acc: 0.8136

[Epoch 21/100]


Train Loss: 0.0977 | Train Acc: 0.9703 | Val Loss: 0.8245 | Val Acc: 0.8136
Early Stopping Triggered!

Best Accuracy: 0.8701


In [4]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Hybrid Model (EfficientNet + VGG19)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()
        
        # ✅ VGG19 사용
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG19 출력 채널

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.vgg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Linear(efficientnet_out_features + vgg_out_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = eff_features.view(eff_features.size(0), -1)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = vgg_features.view(vgg_features.size(0), -1)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)
        return self.classifier(fused_features)

# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 1.0503 | Train Acc: 0.6158 | Val Loss: 1.4668 | Val Acc: 0.5876
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 0.3651 | Train Acc: 0.8927 | Val Loss: 0.8040 | Val Acc: 0.7797
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.1942 | Train Acc: 0.9449 | Val Loss: 0.5780 | Val Acc: 0.8136
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.1545 | Train Acc: 0.9520 | Val Loss: 0.6979 | Val Acc: 0.8305
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.1582 | Train Acc: 0.9590 | Val Loss: 0.9919 | Val Acc: 0.7062

[Epoch 18/100]


Train Loss: 0.1643 | Train Acc: 0.9590 | Val Loss: 0.8751 | Val Acc: 0.7571

[Epoch 19/100]


Train Loss: 0.1757 | Train Acc: 0.9534 | Val Loss: 2.5621 | Val Acc: 0.5198

[Epoch 20/100]


Train Loss: 0.1871 | Train Acc: 0.9463 | Val Loss: 0.8008 | Val Acc: 0.8136

[Epoch 21/100]


Train Loss: 0.0977 | Train Acc: 0.9703 | Val Loss: 0.8245 | Val Acc: 0.8136
Early Stopping Triggered!

Best Accuracy: 0.8701
